In [1]:
import pandas as pd
import time
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch
from torch.utils.data import random_split

In [2]:
data_df = pd.read_pickle('data_df_2015-2020.pickle')
data_df

true-shooting-percentage  defensive-efficiency  \
2014-11-01 Atlanta                         115.8                 1.108   
           Boston                          122.3                 1.013   
           Brooklyn                        113.7                 1.168   
           Charlotte                        94.1                 0.981   
           Chicago                         115.6                 0.970   
...                                          ...                   ...   
2019-03-31 Sacramento                      114.7                 1.194   
           San Antonio                     110.9                 1.063   
           Toronto                         121.5                 0.950   
           Utah                            125.8                 1.007   
           Washington                      110.0                 1.168   

                        average-margin-thru-3-quarters  \
2014-11-01 Atlanta                               -15.0   
           Boston                                 29.0   
           Brooklyn                              -29.0   
           Charlotte                             -15.0   
           Chicago                                 7.5   
...                                                ...   
2019-03-31 Sacramento                             -1.7   
           San Antonio                             4.3   
           Toronto                                18.0   
           Utah                                   10.7   
           Washington                             -5.7   

                        opponent-4th-quarter-points-per-game  \
2014-11-01 Atlanta                                      23.0   
           Boston                                       33.0   
           Brooklyn                                     20.0   
           Charlotte                                    17.0   
           Chicago                                      20.0   
...                                                      ...   
2019-03-31 Sacramento                                   29.3   
           San Antonio                                  25.7   
           Toronto                                      28.3   
           Utah                                         27.0   
           Washington                                   31.3   

                        offensive-efficiency  opponent-shooting-pct  \
2014-11-01 Atlanta                     1.037                   41.1   
           Boston                      1.168                   48.8   
           Brooklyn                    1.013                   55.7   
           Charlotte                   0.999                   48.8   
           Chicago                     1.060                   38.1   
...                                      ...                    ...   
2019-03-31 Sacramento                  1.161                   49.6   
           San Antonio                 1.115                   45.1   
           Toronto                     1.133                   41.4   
           Utah                        1.172                   42.8   
           Washington                  1.109                   52.9   

                        average-scoring-margin  \
2014-11-01 Atlanta                        -7.0   
           Boston                         16.0   
           Brooklyn                      -16.0   
           Charlotte                       2.0   
           Chicago                         9.0   
...                                        ...   
2019-03-31 Sacramento                     -3.3   
           San Antonio                     5.3   
           Toronto                        19.0   
           Utah                           17.3   
           Washington                     -6.3   

                        opponent-defensive-rebounding-pct  \
2014-11-01 Atlanta                                   76.2   
           Boston                                    76.9   
           Brooklyn             

In [3]:
games_df = pd.read_pickle('games_df_2015-2020.pickle')
games_df

,Date,Home,Away,DidHomeWin
0,2016-11-01,Cleveland,Houston,True
1,2016-11-01,Indiana,LA Lakers,True
2,2016-11-01,Philadelphia,Orlando,False
3,2016-11-01,Detroit,New York,True
4,2016-11-01,Miami,Sacramento,True
...,...,...,...,...
6248,2016-03-31,Houston,Chicago,False
6249,2016-03-31,Indiana,Orlando,False
6250,2016-03-31,New Orleans,Denver,True
6251,2016-03-31,Okla City,LA Clippers,True


In [4]:
training_data = []
for i in range(len(games_df)):
    try:
        game = games_df.loc[i]

        homeTeam, awayTeam = str(game.Home), str(game.Away)
        date_str = str(game.Date.date())
        home_index = (date_str, homeTeam) # Index to get the stats for a team on a certain date from data_df
        away_index = (date_str, awayTeam)
        home_stats = data_df.loc[home_index].to_numpy()
        away_stats = data_df.loc[away_index].to_numpy()
        # array_to_append = np.array([ np.array([home_stats.to_numpy(), away_stats.to_numpy()]), game.DidHomeWin])
        # stats = np.array([home_stats.to_numpy(), away_stats.to_numpy()]) # make a 2d array for model eventually
        training_data.append([home_stats, away_stats, int(game.DidHomeWin)]) # [1, 0 ] if AWAY WON,and [0,1] if HOME WON
    except KeyError as e:
        print(date_str) 
        print(home_index)

)
2019-12-25
('2019-12-25', 'Philadelphia')
2019-12-25
('2019-12-25', 'Golden State')
2019-12-25
('2019-12-25', 'LA Lakers')
2019-12-25
('2019-12-25', 'Denver')
2019-12-26
('2019-12-26', 'Detroit')
2019-12-26
('2019-12-26', 'Brooklyn')
2019-12-26
('2019-12-26', 'Dallas')
2019-12-26
('2019-12-26', 'Okla City')
2019-12-26
('2019-12-26', 'Sacramento')
2019-12-26
('2019-12-26', 'Utah')
2019-12-27
('2019-12-27', 'Boston')
2019-12-27
('2019-12-27', 'Charlotte')
2019-12-27
('2019-12-27', 'Orlando')
2019-12-27
('2019-12-27', 'Atlanta')
2019-12-27
('2019-12-27', 'Miami')
2019-12-27
('2019-12-27', 'Golden State')
2019-12-28
('2019-12-28', 'Denver')
2019-12-28
('2019-12-28', 'Boston')
2019-12-28
('2019-12-28', 'New Orleans')
2019-12-28
('2019-12-28', 'Chicago')
2019-12-28
('2019-12-28', 'Houston')
2019-12-28
('2019-12-28', 'Miami')
2019-12-28
('2019-12-28', 'Minnesota')
2019-12-28
('2019-12-28', 'Washington')
2019-12-28
('2019-12-28', 'Golden State')
2019-12-28
('2019-12-28', 'Milwaukee')
2019-12

In [5]:
training_data[0]

[array([107.2  ,   0.914,  13.3  ,  28.3  ,   1.039,  40.2  ,  12.7  ,
         75.   ,  31.   ,  69.   ,  25.   ]),
 array([114.4  ,   1.042,   8.7  ,  27.7  ,   1.052,  44.3  ,   1.   ,
         78.2  ,  22.1  ,  77.9  ,  21.8  ]),
 1]

In [9]:
num_data_points = len(training_data[0][0])*2
x_vals = [x[0:2] for x in training_data]
x_vals = torch.FloatTensor(x_vals)
x_vals = x_vals.view(-1, num_data_points)
y_vals = [y[2] for y in training_data]
y_vals = torch.LongTensor(y_vals)
print(x_vals[0], y_vals[0])
x_vals.shape

tensor([107.2000,   0.9140,  13.3000,  28.3000,   1.0390,  40.2000,  12.7000,
         75.0000,  31.0000,  69.0000,  25.0000, 114.4000,   1.0420,   8.7000,
         27.7000,   1.0520,  44.3000,   1.0000,  78.2000,  22.1000,  77.9000,
         21.8000]) tensor(1)


torch.Size([5342, 22])

In [10]:
dataset = TensorDataset(x_vals, y_vals)

In [11]:
len(dataset) - 500

4842

In [12]:
train_ds, val_ds = random_split(dataset, [4842, 500])

In [13]:
batch_size = 16
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

In [95]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 32) 
        self.fc2 = nn.Linear(32,2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [96]:
for data, win in train_loader:
    print(data)
    print(win)
    net = Net()
    outputs = net(data)
    print(outputs)
    break

tensor([[109.9000, 100.2000,  91.3000, 102.1000],
        [100.0000, 100.3000, 104.6000, 102.1000],
        [100.2000, 102.4000, 103.3000, 104.2000],
        [103.0000, 107.3000, 101.7000,  90.9000],
        [107.8000, 101.8000, 105.1000, 105.1000],
        [ 95.0000, 101.7000,  98.2000,  99.2000],
        [104.2000, 100.8000, 115.3000, 113.4000],
        [107.6000, 100.2000, 106.5000,  97.9000],
        [105.7000, 108.9000, 110.5000, 110.8000],
        [ 99.8000,  95.9000, 107.2000,  96.2000],
        [106.5000,  98.2000, 107.7000, 115.6000],
        [111.3000, 109.8000,  95.3000,  99.6000],
        [ 96.2000,  99.8000, 107.4000, 104.6000],
        [104.0000, 105.0000, 110.7000, 107.9000],
        [106.5000, 106.0000, 107.8000, 107.7000],
        [111.9000, 112.2000, 115.5000, 105.8000]])
tensor([1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1])
tensor([[  0.3420, -20.0126],
        [ -1.1329, -17.5116],
        [ -1.3408, -17.8574],
        [  0.3971, -17.4969],
        [ -0.4758, -18

In [98]:
probs = F.softmax(outputs, dim=1)
probs
max_probs, preds = torch.max(probs, dim = 1)
print(preds)
torch.sum(preds == win)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


tensor(6)

In [15]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [16]:
loss_fn = F.cross_entropy

In [17]:
loss = loss_fn(outputs, win)
print(loss)

NameError: name 'outputs' is not defined

In [18]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [19]:
def fit(epochs, learning_rate, model, train_loader, val_loader, opt_func=torch.optim.SGD): # mess around with optimizer
    optimizer = opt_func(model.parameters(), learning_rate)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [23]:
class BBallNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(22, 64) 
        self.fc2 = nn.Linear(64,2)
        
    def forward(self, data):
        out = self.fc1(data)
        out = F.relu(out)
        out = self.fc2(out)
        return out
    
    def training_step(self, batch):
        data, win_labels = batch 
        out = self(data)                  # Generate predictions
        loss = F.cross_entropy(out, win_labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [24]:
BBModel = BBallNet()

In [28]:
history1 = fit(10, 0.01, BBModel, train_loader, val_loader)

Epoch [0], val_loss: 0.6849, val_acc: 0.5586
Epoch [1], val_loss: 0.6851, val_acc: 0.5586
Epoch [2], val_loss: 0.6805, val_acc: 0.5645
Epoch [3], val_loss: 0.6867, val_acc: 0.5586
Epoch [4], val_loss: 0.6752, val_acc: 0.5762
Epoch [5], val_loss: 0.6831, val_acc: 0.5645
Epoch [6], val_loss: 0.6784, val_acc: 0.5742
Epoch [7], val_loss: 0.6953, val_acc: 0.5547
Epoch [8], val_loss: 0.6929, val_acc: 0.5566
Epoch [9], val_loss: 0.6950, val_acc: 0.5566


In [ ]:
def determine_pythag_expectation(points_against, points_for):
    win = (points_for**(13.91)) / (points_for**13.91 + points_against**13.91)
    return win
def get_point_differential(points_against, points_for):
    return points_for-points_against

In [ ]:
correct = 0
extremly_wrong = 0
for data in training_data:
    home_expectation = determine_pythag_expectation(data[0][0], data[0][1])
    away_expectation = determine_pythag_expectation(data[1][0], data[1][1])
    #print(f"Home: {home_expectation} Away :{away_expectation} HomeWin: {data[2]}")
    if (data[0][1] >= data[1][1]) and data[2]:
        correct += 1
    else:
        extremly_wrong += 1

print(correct)
print(extremly_wrong)
print(correct/(correct+extremly_wrong))

In [ ]:
home_expectation

In [ ]:
# building the conv net
training_data[0]

In [ ]:
net = Net()
# pass 

In [ ]:
X = torch.randn((2,2))
X.view(-1, 4)

In [ ]:
X = torch.Tensor[i[0]]